In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import vis
import evaluate
from utils import *

from collections import Counter
from pprint import pprint
from functools import reduce

import re
import numpy as np
from nltk.tree import Tree
from nltk import word_tokenize,sent_tokenize

not use stanford CoreNLP client!


[nltk_data] Error loading sentiwordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>


In [2]:
# is_news = 1

# l = 7
# t = 4
# m = "c_v"
# i = 200
# min_df = 1

is_news = 0
l = 6
t = 14
m = "c_v"
i = 200
min_df = 1

# load
if is_news:
    _raw = persister.load_json(configs.RAWNEWS)
    parse = persister.read_parse()
    _input = persister.read_input(configs.NEWSINPUT)
    model_name = configs.NEWSMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA.format(model_name))
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.NEWSVEC.format(min_df))
    print("load news")
else:
    _raw = persister.load_json(configs.RAWABSTRACT)
    _input = persister.read_input(configs.ABSTRACTINPUT)
    model_name = configs.ABSTRACTMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA.format(model_name))
    parse = persister.read_parse(configs.ABSTRACTPARSE)
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.ABSVEC.format(min_df))
    print("load abs")
tf = vec.fit_transform(_input)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)
df_top_words, df_top_docs = lda.pd_topics_vis(top_terms, top_docs)

load abs


In [3]:
len(terms)

10453

In [4]:
distr = lda.get_dominant_topic(doc_topic)
print("docs distribution:",dict(Counter(distr)))

docs distribution: {9: 152, 5: 33, 11: 969, 12: 1330, 1: 7, 13: 3, 10: 76, 3: 3, 0: 8, 7: 14, 6: 1}


In [5]:
print("top terms info:")
df_top_words

top terms info:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,"(3730, 123.1)","(2699, 96.87)","(3253, 78.19)","(2522, 77.49)","(7395, 75.45)","(895, 64.39)","(6752, 51.34)","(1245, 51.19)","(4333, 48.51)","(888, 47.03)","(1071, 40.99)","(107, 40.76)","(8958, 39.19)","(466, 37.74)","(5491, 31.56)","(5967, 29.42)","(7771, 29.31)","(1696, 28.59)","(9501, 26.52)","(7766, 25.93)"
Topic 1,"(6755, 223.06)","(5621, 143.69)","(4041, 143.5)","(1426, 139.72)","(9624, 99.44)","(2398, 86.78)","(2522, 81.07)","(8012, 79.0)","(7666, 78.67)","(1171, 68.06)","(3624, 47.64)","(4042, 42.31)","(1146, 39.36)","(1490, 36.28)","(2161, 32.74)","(1696, 30.77)","(6926, 29.99)","(2935, 25.48)","(2836, 25.13)","(2633, 24.57)"
Topic 2,"(5788, 42.64)","(6839, 25.17)","(7529, 13.82)","(4168, 9.76)","(256, 8.0)","(7408, 7.74)","(5407, 7.74)","(5085, 7.05)","(599, 6.96)","(1303, 6.65)","(10434, 6.65)","(5137, 6.22)","(7265, 5.86)","(5844, 5.86)","(4352, 5.13)","(1730, 5.01)","(9584, 5.0)","(6774, 4.14)","(9263, 4.12)","(8352, 4.12)"
Topic 3,"(2920, 181.73)","(2689, 87.04)","(8961, 86.19)","(8739, 55.34)","(3757, 39.85)","(7653, 38.09)","(8708, 34.84)","(8847, 33.33)","(10381, 31.62)","(5081, 30.16)","(84, 30.1)","(2693, 29.08)","(2691, 27.25)","(3082, 26.57)","(8560, 24.96)","(10078, 24.49)","(684, 22.67)","(6250, 22.09)","(674, 21.59)","(2533, 21.36)"
Topic 4,"(8129, 79.5)","(6679, 73.15)","(3803, 67.68)","(8271, 57.19)","(2579, 52.59)","(1527, 42.46)","(1455, 33.26)","(1003, 32.4)","(1021, 31.0)","(5090, 30.54)","(2018, 29.71)","(8165, 29.68)","(9489, 29.34)","(481, 28.55)","(5646, 27.44)","(3858, 27.32)","(8128, 26.77)","(5551, 23.32)","(620, 23.12)","(5719, 22.88)"
Topic 5,"(7969, 537.88)","(8339, 474.31)","(6502, 265.32)","(9716, 235.55)","(7297, 228.45)","(7103, 206.48)","(6471, 196.94)","(9508, 194.3)","(350, 180.66)","(9724, 174.52)","(5615, 171.56)","(559, 166.01)","(2109, 159.65)","(7581, 142.65)","(1812, 111.75)","(8626, 101.7)","(9991, 101.33)","(6138, 88.63)","(7096, 85.19)","(8958, 81.34)"
Topic 6,"(8659, 81.12)","(651, 74.02)","(6058, 57.13)","(7010, 35.01)","(1051, 28.77)","(6479, 24.22)","(4526, 19.81)","(6059, 18.49)","(5409, 17.49)","(3092, 16.39)","(10382, 14.97)","(1578, 14.53)","(8986, 13.59)","(4285, 11.69)","(2814, 10.71)","(5957, 10.39)","(4869, 9.79)","(7954, 9.73)","(7831, 9.62)","(9448, 8.87)"
Topic 7,"(8639, 268.18)","(7518, 203.92)","(1490, 149.7)","(2376, 143.61)","(8669, 143.52)","(1076, 139.43)","(7834, 120.26)","(7336, 87.55)","(7322, 68.98)","(411, 65.81)","(7840, 53.22)","(7865, 49.86)","(3137, 49.8)","(9280, 48.97)","(6483, 44.34)","(8958, 43.24)","(5388, 40.78)","(8810, 33.35)","(1296, 30.56)","(6211, 30.05)"
Topic 8,"(5310, 75.09)","(3592, 55.04)","(5228, 31.56)","(562, 28.23)","(8928, 22.76)","(5151, 18.42)","(158, 18.41)","(8022, 17.93)","(5, 15.75)","(172, 15.54)","(113, 15.29)","(5389, 13.09)","(2882, 13.06)","(4503, 12.3)","(641, 11.6)","(8121, 11.02)","(10215, 10.98)","(3877, 10.91)","(4946, 10.27)","(2625, 10.12)"
Topic 9,"(9508, 893.39)","(7874, 662.45)","(8958, 393.56)","(350, 331.47)","(9991, 254.09)","(533, 239.38)","(4333, 225.84)","(9286, 215.99)","(9632, 201.75)","(5296, 154.3)","(3409, 150.61)","(9344, 137.1)","(6669, 135.72)","(5108, 131.54)","(6739, 131.04)","(7876, 130.56)","(8204, 129.8)","(505, 125.64)","(2468, 123.53)","(292, 122.98)"


In [6]:
print("top docs info:")
df_top_docs

top docs info:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Topic 0,"(198, 0.548)","(263, 0.493)","(107, 0.407)","(1764, 0.386)","(2009, 0.383)","(1770, 0.371)","(1740, 0.361)","(894, 0.346)","(896, 0.334)","(65, 0.32)","(751, 0.318)","(725, 0.318)","(122, 0.314)","(52, 0.285)","(2577, 0.271)","(562, 0.264)","(526, 0.263)","(50, 0.257)","(685, 0.247)","(2248, 0.245)"
Topic 1,"(489, 0.751)","(142, 0.651)","(115, 0.587)","(722, 0.42)","(1566, 0.37)","(1101, 0.364)","(1112, 0.346)","(80, 0.346)","(32, 0.345)","(627, 0.339)","(415, 0.319)","(2304, 0.319)","(1037, 0.313)","(1446, 0.291)","(1402, 0.291)","(1129, 0.281)","(174, 0.277)","(64, 0.276)","(338, 0.275)","(1624, 0.266)"
Topic 2,"(1855, 0.23)","(1319, 0.208)","(1400, 0.199)","(872, 0.198)","(1309, 0.196)","(1079, 0.187)","(1340, 0.151)","(1310, 0.146)","(1164, 0.133)","(1350, 0.131)","(2158, 0.115)","(2386, 0.114)","(2188, 0.112)","(1242, 0.11)","(2220, 0.107)","(1868, 0.105)","(1932, 0.104)","(2422, 0.102)","(2457, 0.075)","(1876, 0.074)"
Topic 3,"(81, 0.685)","(874, 0.517)","(109, 0.374)","(771, 0.349)","(18, 0.338)","(195, 0.293)","(1178, 0.287)","(344, 0.285)","(1281, 0.265)","(1813, 0.255)","(171, 0.254)","(1422, 0.254)","(1761, 0.244)","(1306, 0.24)","(1266, 0.238)","(737, 0.233)","(1278, 0.228)","(113, 0.222)","(1175, 0.22)","(365, 0.216)"
Topic 4,"(508, 0.371)","(461, 0.346)","(413, 0.333)","(98, 0.282)","(1076, 0.27)","(720, 0.268)","(740, 0.268)","(741, 0.268)","(1666, 0.252)","(606, 0.248)","(693, 0.247)","(2027, 0.247)","(1379, 0.246)","(82, 0.241)","(2447, 0.231)","(886, 0.226)","(89, 0.225)","(1273, 0.217)","(450, 0.215)","(56, 0.206)"
Topic 5,"(85, 0.951)","(57, 0.727)","(386, 0.669)","(111, 0.651)","(1, 0.631)","(575, 0.57)","(434, 0.518)","(698, 0.513)","(1577, 0.493)","(786, 0.486)","(103, 0.48)","(59, 0.471)","(6, 0.466)","(270, 0.455)","(957, 0.454)","(2272, 0.452)","(67, 0.452)","(973, 0.451)","(410, 0.435)","(1936, 0.42)"
Topic 6,"(1570, 0.465)","(1117, 0.415)","(1606, 0.337)","(477, 0.302)","(757, 0.276)","(970, 0.254)","(1874, 0.252)","(1167, 0.169)","(1520, 0.166)","(684, 0.162)","(1668, 0.15)","(854, 0.147)","(2526, 0.144)","(702, 0.141)","(581, 0.141)","(1021, 0.131)","(2325, 0.126)","(2396, 0.125)","(1720, 0.124)","(995, 0.12)"
Topic 7,"(1586, 0.575)","(747, 0.526)","(224, 0.503)","(831, 0.447)","(453, 0.428)","(898, 0.427)","(20, 0.397)","(1594, 0.396)","(731, 0.392)","(2283, 0.386)","(380, 0.385)","(1473, 0.382)","(891, 0.379)","(1181, 0.377)","(2312, 0.376)","(2533, 0.375)","(945, 0.371)","(2467, 0.368)","(752, 0.357)","(141, 0.357)"
Topic 8,"(441, 0.215)","(1958, 0.173)","(890, 0.172)","(626, 0.172)","(2121, 0.167)","(2184, 0.165)","(70, 0.161)","(802, 0.16)","(1442, 0.157)","(273, 0.152)","(2561, 0.148)","(1343, 0.147)","(1440, 0.137)","(576, 0.136)","(2505, 0.129)","(2233, 0.125)","(1882, 0.123)","(849, 0.114)","(2522, 0.113)","(621, 0.108)"
Topic 9,"(2, 0.994)","(41, 0.9)","(1044, 0.871)","(317, 0.861)","(1013, 0.836)","(97, 0.813)","(996, 0.8)","(47, 0.798)","(1475, 0.772)","(335, 0.772)","(1305, 0.772)","(388, 0.765)","(308, 0.755)","(120, 0.752)","(546, 0.741)","(51, 0.737)","(8, 0.727)","(617, 0.724)","(264, 0.716)","(564, 0.712)"


In [7]:
# vis.pyLDA(topic_word, doc_topic, [len(s) for s in [word_tokenize(corp) for corp in _input]], vec.get_feature_names(), np.array(sum(tf).todense())[0])

In [8]:
t_idx = 1
for top_doc in top_docs[t_idx]:
    _idx = top_doc[0]
    if type(parse[_idx]) is str:
        print(_idx, "parse", "err")
        continue
    sents = relation.convert_parse2lemma_sents(parse[_idx])
    sort_idxs, importance, counts = relation.extract_important_sents(sents, [terms[x[0]] for x in top_terms[t_idx]], [x[1] for x in top_terms[t_idx]])
    for i in sort_idxs:
        if importance[i]>0:
            print(round(importance[i], 2), counts[i], _idx, i)
            sent_tokens = parse[_idx]["sentences"][i]["tokens"]
            sent_deps = parse[_idx]["sentences"][i]["enhancedPlusPlusDependencies"]
            print(" ".join([s["lemma"] for s in sent_tokens]))
            sents = sent_tokenize(_raw[_idx])
            if len(sents) > i:
                print(sents[i])
            else:
                print("sent token differ")
            triples = relation.extract_triples_from_sent(sent_deps, sent_tokens, True)
            for triple in triples:
                if None in triple:
                    print(triple)
                    continue
                s = [sent_tokens[i]["originalText"] for i in triple[0]]
                p = [sent_tokens[i]["originalText"] for i in triple[1]]
                o = [sent_tokens[i]["originalText"] for i in triple[2]]
                print("{}-{}-{}".format(s,p,o))
            print("="*40)

1.07 2 489 5
lda show that , prior to surgery , patient ' priority be primarily in cancer surgery and recovery .
LDA showed that, prior to surgery, patients' priorities were primarily in cancer surgery and recovery.
[(0,), (1,), None]
['priorities']-['were']-['cancer', 'surgery', 'and', 'recovery']
[(0,), (17,), None]
1.07 2 489 4
we aim to apply LDA to interview datum collect as part of a prospective , longitudinal study of qol in patient undergo radical cystectomy and urinary diversion for bladder cancer .
We aim to apply LDA to interview data collected as part of a prospective, longitudinal study of QOL in patients undergoing radical cystectomy and urinary diversion for bladder cancer.
[(0,), (1,), None]
['We']-['apply']-['LDA']
['We']-['interview']-['data']
1.0 1 489 3
latent Dirichlet allocation -lrb- lda -rrb- may offer statistical rigor in summarize patient ' concern and cope strategy in a life-threatening illness .
Latent Dirichlet Allocation (LDA) may offer statistical rigor i

0.06 1 1101 14
beside these , the prediction for procedure code base on the corr-wddcrf also show considerable accuracy .
Beside these, the prediction for procedure codes based on the Corr-wddCRF also shows considerable accuracy.
['prediction', 'for', 'codes']-['shows']-['considerable', 'accuracy']
1.4 3 1112 0
information and communication technology have enable healthcare institution to accumulate large amount of healthcare datum that include diagnosis , medication , and additional contextual information such as patient demographic .
Information and communications technologies have enabled healthcare institutions to accumulate large amounts of healthcare data that include diagnoses, medications, and additional contextual information such as patient demographics.
['Information', 'communications', 'technologies']-['enabled']-['healthcare', 'institutions']
['healthcare', 'institutions']-['accumulate']-['large', 'amounts', 'of', 'data']
0.98 3 1112 1
to gain a better understanding of big

Recently, with the rapid development of hospital information systems, a large volume of electronic medical records (EMRs) has been produced, which provides a comprehensive source for CP analysis.
['large', 'volume', 'of', 'records']-['produced']-['rapid', 'development', 'of', 'systems']
0.38 2 1446 5
experimental result on a real collection of 985 emr collect from a chinese hospital show that the propose approach can effectively identify meaningful treatment pattern from emr .
Experimental results on a real collection of 985 EMRs collected from a Chinese hospital show that the proposed approach can effectively identify meaningful treatment patterns from EMRs.
[(0, 1, 2, 5), (14,), None]
['proposed', 'approach']-['identify']-['meaningful', 'treatment', 'patterns', 'from', 'EMRs']
0.0 1 1446 2
in this paper , we be concern with the problem of utilize the heterogeneous emr to assist cp analysis and improvement .
In this paper, we are concerned with the problem of utilizing the heterogeneo

0.03 1 338 6
mixed method , include a literature review , data-driven-based topic discovery , and human annotation be use to develop the physician review topic taxonomy .
Mixed methods, including a literature review, data-driven-based topic discovery, and human annotation were used to develop the physician review topic taxonomy.
[(0, 1), (16,), None]
['Mixed', 'methods']-['develop']-['physician', 'review', 'topic', 'taxonomy']
0.03 2 338 5
method : datum comprise 122,716 physician review , include review of 8501 doctor from a lead physician review website in China -lrb- haodf.com -rrb- , collect between 2007 and 2015 .
Methods: Data comprised 122,716 physician reviews, including reviews of 8501 doctors from a leading physician review website in China (haodf.com), collected between 2007 and 2015.
0.03 1 338 2
the first step toward mining physician review be to determine how the natural structure or dimension be embed in review .
The first step toward mining physician reviews is to deter

In [9]:
extended = relation.extend_lda_results(parse, _input, top_terms, top_docs, terms, "lemma", top_n=-1, score_method="tf")
print(len(extended[t_idx]))
for triple in extended[t_idx]:
    print(round(triple[1],2),"{}-[{}]-{}".format(" ".join(triple[0][0])," ".join(triple[0][1])," ".join(triple[0][2])))

158 parse err
208
175 traditional topic model-[discover]-disease topic
155 latent Dirichlet allocation topic model-[fit to]-data set
148 traditional topic model-[discover]-emr datum
148 latent Dirichlet allocation topic model-[explore]-unsupervised manner
147 exist risk stratification technique-[predict]-overall risk of patient
143 many datum mining technique-[discover]-latent treatment information for analysis
141 hospital emr dataset-[consist of]-medical record of patient
139 patient record-[contain]-valuable medical information
139 Learning base deep learning method-[propose in]-heart disease diagnosis
132 traditional topic model-[treat]-document and code as word
129 allocation topic and vector-[use as]-multilabel classification method
129 allocation topic and vector-[use as]-feature and method
126 mixed method-[develop]-physician review topic taxonomy
122 result-[classify]-heart disease and patient
122 additional implicit patient need-[use]-qualitative analysis
121 present study-[a

In [ ]:
ws = 20
measure = "c_uci"
old = lda.get_coherence(tf, terms, topic_word, _input, measure, window_size=ws)
fake_idxs = []
min_top = 999
for e in extended: # topic
    new_word_idxs = []
    for t in e:# triples
         new_word_idxs.extend(evaluate.filter_triple2term_idx(t[0], vec))
    if min_top > len(set(new_word_idxs)):
        min_top = len(set(new_word_idxs))
    fake_idxs.append(set(new_word_idxs))
fake_topic_word = evaluate.gen_fake_topic_word(topic_word.shape, fake_idxs)
new = lda.get_coherence(tf, terms, fake_topic_word, _input, measure,min_top,window_size=ws)
print(old, new)
print(np.mean(old), np.mean(new))

In [ ]:
print([terms[i] for i in fake_idxs[1]])
print([terms[i[0]] for i in top_terms[1]])

In [ ]:
ps = relation.get_phrases_by_pattern(parse)
sorted(dict(Counter([" ".join(x) for x in ps])).items(), key=lambda x:x[1], reverse=True)